## An Exploration of Chicago's Array of Things API (via python client)
Array of Things has expanded in Chicago to include more than 100 continuous sensors across the city.

**Resources**
* [Home Page](https://arrayofthings.github.io)
* [python client](https://github.com/UrbanCCD-UChicago/aot-client-py)
* [API documentation](https://arrayofthings.docs.apiary.io)

**TODOs**
  * group measurements by larger cluster than sensor? by project?
  * efficiently query regular interval of data, to annimate
  * normalize measurements, choose reasonable circle radii and color
    * map different sensors to different visualizations

In [ ]:
 # !pip install aot-client

In [ ]:
from aot_client import AotClient

client = AotClient()

# What are the methods/properties of the client?
[_ for _ in dir(client) if not _.startswith('_')]

It appears results will be paginated and we can also pass a filter. From the docs website I see that users may pass a timestamp filter in ISO 8601 format. Let's get measurements from the past 15 minutes.

In [ ]:
import datetime

import pandas as pd
from aot_client import AotClient
from aot_client import F


def time_x_mins_ago(minutes:int):
    '''Get formatted time to pass to API filter, relative to current time
    '''
    t = (datetime.datetime.now() - 
         datetime.timedelta(minutes=minutes) + 
         datetime.timedelta(hours=5))  # convert timezone from central to UTC
    t = t.isoformat()
    
    return t[0:19]


def unpack_response(response, page_limit=1000):
    try:
        pages = []
        for i, page in enumerate(response):
            if i + 1 > page_limit:
                break
            pages.extend(page.data)
    except HTTPError as e:
        print(e)    
    finally:
        return pages


def process_observations(obs_df):
    obs_df = obs_df.copy()
    obs_df['timestamp'] = pd.to_datetime(obs_df['timestamp'], utc=True)
    obs_df['timestamp'] = obs_df['timestamp'].dt.tz_convert('US/Central')
    
    # extract lat/lon to columns
    obs_df['coords'] = obs_df['location'].apply(
        lambda x: x['geometry']['coordinates'])
    obs_df[['lon', 'lat']] = pd.DataFrame(
        obs_df['coords'].tolist(), columns=['lon', 'lat'])
    obs_df = obs_df.drop(columns=['coords'])
    
    # fix positive lon values
    mask = obs_df['lon'] > 0
    if sum(mask) > 0:
        print(f'fixed {sum(mask)} rows with positive lon value')
        obs_df.loc[mask, 'lon'] = obs_df.loc[mask, 'lon'] * -1

    # remove lat/lon values at 0
    mask = (obs_df['lon'] != 0) & (obs_df['lat'] != 0)
    if len(obs_df) - sum(mask) > 0:
        print(f'removed {len(obs_df) - sum(mask)} rows with lat/lon at 0')
        obs_df = obs_df.loc[mask]

    # remove lat values less than 40 degrees
    mask = (obs_df['lat'] > 40)
    if len(obs_df) - sum(mask) > 0:
        print(f'removed {len(obs_df) - sum(mask)} '
              'rows with lat/lon outside Chicago region')
        obs_df = obs_df.loc[mask]
    
    return obs_df

In [ ]:
client = AotClient()

# create filter
f = F('size', '90000')
f &= ('timestamp', 'ge', time_x_mins_ago(5))
# f &= ('sensor', 'image.image_detector.person_total')
# f &= ('time_bucket', 'avg:1 hour')
# f &= ('sensor', 'image.image_detector.car_total')
# f &= ('sensor', 'metsense.tsys01.temperature')

response = client.list_observations(filters=f)
print(response.current_link)
pages = unpack_response(response, page_limit=1)
print(len(pages))
obs_df = pd.DataFrame(pages)
# obs_df = process_observations(obs_df)
print(f"shape: {obs_df.shape}")
obs_df.head()

In [ ]:
len(obs_df['node_vsn'].unique())

In [ ]:
obs_df.groupby('node_vsn')['sensor_path'].nunique()

In [ ]:
import folium
import folium.plugins

def map(df):
    m = folium.Map(location=[df['lat'].mean(), 
                             df['lon'].mean()],
                   tiles='CartoDB dark_matter',
                   zoom_start=10)

    for i, r in df.iterrows():
        folium.CircleMarker(
            location=(r['lat'], r['lon']),              
#             radius=3,
#             color=r['color'],
#             weight=0.5,
            tooltip=f"{r['timestamp']}<br>{r['value']} {r['uom']}",
#             popup=folium.Popup(f"{r['value']} {r['uom']}", max_width=500),
            fill=True
        ).add_to(m)

    folium.plugins.Fullscreen(
        position='topright',
        force_separate_button=True
    ).add_to(m)

    return m

In [ ]:
response.current_link

In [ ]:
len(sensors_df['path'].unique())

In [ ]:
obs_df.head()

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

sensors = client.list_sensors()
sensors_df = pd.DataFrame(sensors.data)
sensors_df.head()

@interact_manual
def choose_sensor(sensor=sensors_df['path'].unique()):
    client = AotClient()
    f = F('sensor', sensor)

    response = client.list_observations(filters=f)
    print('API call:', response.current_link)
    pages = unpack_response(response, page_limit=5)
    
    if not pages:
        print('No data found.')
        return None
    
    obs_df = pd.DataFrame(pages)
    obs_df = process_observations(obs_df)
    print(obs_df.shape)
    
    return map(obs_df.drop_duplicates(['lon', 'lat']))

### Plotly

In [ ]:
# initial configuration
import plotly
from os import getenv
from dotenv import load_dotenv

load_dotenv()

plotly.tools.set_credentials_file(
    username=getenv('PLOTLY_USER'), 
    api_key=getenv('PLOTLY_API_KEY')
)

In [ ]:
# exploring plots
import plotly.plotly as py
import plotly.graph_objs as go
from IPython.display import IFrame

trace0 = go.Scatter(
    x=[1, 2, 3, 4],
    y=[10, 15, 13, 17]
)
trace1 = go.Scatter(
    x=[1, 2, 3, 4],
    y=[16, 5, 11, 9]
)
data = [trace0, trace1]

plot_url = py.plot(data, filename = 'basic-line', auto_open=False)
IFrame(src=plot_url, width="100%", height="600px", frameBorder="0")

In [ ]:
import pandas as pd
SENSOR_DF = pd.read_csv('data/sensor_mapping.csv')

In [ ]:
SENSOR_DF.groupby('sensor_type')['sensor_measure'].unique().to_dict()

In [ ]:
def query_aot(sensor_hrf, size_per_page=100000, page_limit=1, mins_ago=12*60):
    sensor = SENSOR_DF.loc[SENSOR_DF['sensor_measure']==sensor_hrf, 
                           'sensor_path'].values[0]

    client = AotClient()

    f = F('size', str(size_per_page))
    f &= ('sensor', sensor)
    f &= ('timestamp', 'ge', time_x_mins_ago(mins_ago))

    response = client.list_observations(filters=f)
    pages = unpack_response(response, page_limit=page_limit)
    obs_df = pd.DataFrame(pages)
    obs_df = process_observations(obs_df)
    
    return obs_df

In [ ]:
df = query_aot('Carbon Monoxide')

## Import data from AoT Archive

In [1]:
import pandas as pd

In [3]:
from app.aot import load_aot_archive_day, clean_aot_archive_obs

In [4]:
df = load_aot_archive_day(day='2019-05-14')
df = clean_aot_archive_obs(df)

C:\Users\City_Year\GitHub\chicago-array-build-ds\app\aot.py:122: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [15]:
df.to_sql?

In [26]:
df['id'] = list(range(max_id + 1, max_id + len(df) + 1))

In [29]:
df = load_aot_archive_day(day='2019-05-14')

In [31]:
df['node_id'].unique()

array(['001e0610bc12', '001e061144d6', '001e0610ba46', '001e0610e835',
       '001e0610eef2', '001e06118295', '001e061130f4', '001e06113cf1',
       '001e06113d22', '001e0611462f', '001e0610ef27', '001e06112e77',
       '001e061146bc', '001e0610e537', '001e0610ee36', '001e0610bc10',
       '001e06113d78', '001e0611536c', '001e0610ee5d', '001e0610fb4c',
       '001e06113ace', '001e061146cb', '001e06109401', '001e0610ba15',
       '001e06113a24', '001e06114fd4', '001e0610ee33', '001e06113107',
       '001e0611537d', '001e0610bbf9', '001e0610f05c', '001e06113ad8',
       '001e061146ba', '001e0610ba16', '001e0610ee43'], dtype=object)

In [33]:
df = clean_aot_archive_obs(df)

In [35]:
df['node_id'].unique()

array(['001e06109401', '001e0610ba15', '001e0610ba16', '001e0610ba46',
       '001e0610bbf9', '001e0610bc10', '001e0610bc12', '001e0610e537',
       '001e0610e835', '001e0610ee33', '001e0610ee36', '001e0610ee43',
       '001e0610ee5d', '001e0610eef2', '001e0610ef27', '001e0610f05c',
       '001e0610fb4c', '001e06112e77', '001e061130f4', '001e06113107',
       '001e06113a24', '001e06113ace', '001e06113ad8', '001e06113cf1',
       '001e06113d22', '001e06113d78', '001e061144d6', '001e0611462f',
       '001e061146ba', '001e061146bc', '001e061146cb', '001e06114fd4',
       '001e0611536c', '001e0611537d', '001e06118295'], dtype=object)